# Learning Python as an R user

# Import libraries

In [12]:
import pandas
import math
import numpy
import plotly.express
import plotly.offline
import scipy
import statsmodels.api
import statsmodels.formula.api
import patsy.contrasts

# Generate data
## `group` variable with 4 levels

In [13]:
group = numpy.repeat(["A", "B", "C", "D"], repeats = 25)

## Save parameters

In [14]:
mean = (3, 4, 4, 3)
sigma = (1, 1, 1, 1)
n = (25, 25, 25, 25)

## Random normal variable `x` whose means depend on levels of `group` variable

In [15]:
# Empty list for appending results
x = []

# Loop through equal-length mean, sigma, and n (group size)
for (i, j, k) in zip(mean, sigma, n):
    x.append(numpy.random.normal(loc = i, scale = j, size = k))

# Concatenate transforms the result into 1 array
x = numpy.concatenate(x)

## Variable `y` correlated r = 0.75 with x

In [16]:
y = x * 0.75 + numpy.random.normal(loc = 0, scale = 1, size = sum(n))

## Store variables in a data frame

In [17]:
data1 = pandas.DataFrame({"y": y, "x": x, "group": group})

## Create contrast variables for use in linear regression on `group` variable

In [18]:
# Helmert contrasts
group_helmert = patsy.contrasts.Helmert().code_without_intercept(list(set(group)))

# 2 main effects and 1 interaction contrast
group_factorial = patsy.contrasts.ContrastMatrix([[-1, -1, 1], [-1, 1, -1], [1, -1, -1], [1, 1, 1]], 
                                                 ["Main Effect 1", "Main Effect 2", "Interaction"])

## Add contrast variables to data frame

In [19]:
data1.loc[data1["group"] == "A", "main1"] = -1
data1.loc[data1["group"] == "B", "main1"] = -1
data1.loc[data1["group"] == "C", "main1"] = 1
data1.loc[data1["group"] == "D", "main1"] = 1

data1.loc[data1["group"] == "A", "main2"] = -1
data1.loc[data1["group"] == "B", "main2"] = 1
data1.loc[data1["group"] == "C", "main2"] = -1
data1.loc[data1["group"] == "D", "main2"] = 1

data1.loc[data1["group"] == "A", "interaction"] = 1
data1.loc[data1["group"] == "B", "interaction"] = -1
data1.loc[data1["group"] == "C", "interaction"] = -1
data1.loc[data1["group"] == "D", "interaction"] = 1

# Plots
## Boxplots

In [20]:
# boxplots of y by group
plotly.offline.iplot(
    plotly.express.box(data1, x = "group", y = "y")
)

In [21]:
# boxplots of x by group
plotly.offline.iplot(
    plotly.express.box(data1, x = "group", y = "x")
)

## Histograms

In [22]:
# y histograms by group
plotly.offline.iplot(
    plotly.express.histogram(data1, x = "y", facet_col = "group")
)

In [23]:
# x histograms by group
plotly.offline.iplot((
    plotly.express.histogram(data1, x = "x", facet_col = "group")
)

SyntaxError: unexpected EOF while parsing (<ipython-input-23-940636e226de>, line 4)

## Scatterplot

In [ ]:
plotly.offline.iplot(
    plotly.express.scatter(data1, x = "x", y = "y", color = "group", trendline = "ols", facet_col = "group")
)

## Bars of `group` means with 95% confidence intervals

In [ ]:
group_desc = data1.groupby("group")["y"].agg(["mean", "sem", "count"]).reset_index()
group_desc["df"] = group_desc["count"] - 1
group_desc["lower"] = group_desc["mean"] - scipy.stats.t.ppf(1 - 0.05 / 2, df = group_desc["df"]) * group_desc["sem"]
group_desc["upper"] = group_desc["mean"] + scipy.stats.t.ppf(1 - 0.05 / 2, df = group_desc["df"]) * group_desc["sem"]

# Plot
plotly.offline.iplot(
    plotly.express.bar(group_desc, x = "group", y = "mean", error_y_minus = "lower", error_y = "upper", color = "group")
)

# Descriptive Statistics

In [ ]:
data1.groupby("group")[["x", "y"]].describe().round(2)

# Correlation

In [ ]:
# Save r and p-value
r1, pvalue1 = scipy.stats.pearsonr(x, y)

# Save degrees of freedom
ddf1 = len(x) - 2

# Compute t-statistic and degress of freedom, isf for upper tail of t distribution
t1 = scipy.stats.t.isf(pvalue1 / 2, df = ddf1)

# Compute standard error
se1 = r1 / t1

# Compute lower and upper confidence intervals
lower1, upper1 = (numpy.tanh(numpy.arctanh(r1) - 1 / math.sqrt(len(x) - 3) * scipy.stats.norm.ppf(1 - 0.05 / 2)),
                  numpy.tanh(numpy.arctanh(r1) + 1 / math.sqrt(len(x) - 3) * scipy.stats.norm.ppf(1 - 0.05 / 2)))

# Print most basic results
"r = {0}, 95%CI [{1}, {2}], t({3}) = {4}, p = {5}".format(r1.round(2), lower1.round(2), upper1.round(2), ddf1, t1.round(2), pvalue1.round(3))

# Analyses
## Regression
### Fit linear regression

In [ ]:
# regress y on numeric/continuous x
ols_fit1 = statsmodels.formula.api.ols("y ~ x", data = data1).fit()

### Results Summary

In [ ]:
ols_fit1.summary()

## Fit linear regression

In [ ]:
# helmert contrasts on group
ols_fit2 = statsmodels.formula.api.ols("y ~ C(group, group_helmert)", data = data1).fit()

### Results Summary

In [ ]:
ols_fit2.summary()

In [ ]:
# factorial contrasts (2 main effects and 1 interaction)
ols_fit3 = statsmodels.formula.api.ols("y ~ C(group, group_factorial)", data = data1).fit()

## Results Summary

In [ ]:
ols_fit3.summary()

## Fit linear regression

In [ ]:
ols_fit4 = statsmodels.formula.api.ols("y ~ main1 + main2 + interaction", data = data1).fit()

## Results Summary

In [ ]:
ols_fit4.summary()